# Hodgkin-Huxley Model: Zap Analysis

This notebook implements and analyzes the Hodgkin-Huxley model's response to a zap current - an oscillating current with linearly increasing frequency. This analysis helps reveal the Type-II behavior of the model through resonance in the ZAP response.

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import matplotlib.pyplot as plt
from src.model import HodgkinHuxleyModel
from src.visualization import plot_membrane_potential

plt.style.use('seaborn-darkgrid')

## 1. Zap Current Implementation

First, let's implement the zap current function that generates an oscillating current with linearly increasing frequency.

In [2]:
def create_zap_current(max_amplitude, fmin, fmax, duration):
    """Create a zap current with linearly increasing frequency.
    
    Args:
        max_amplitude (float): Maximum amplitude of the oscillating current (μA/cm²)
        fmin (float): Initial frequency (Hz)
        fmax (float): Final frequency (Hz)
        duration (float): Total duration of the simulation (ms)
    
    Returns:
        function: Current input function
    """
    def current_func(t):
        # Convert time to seconds for frequency calculation
        t_sec = t / 1000.0
        duration_sec = duration / 1000.0
        
        # Ensure t is within the duration
        if t_sec > duration_sec:
            return 0
        
        # Calculate instantaneous frequency
        df = (fmax - fmin) / duration_sec
        f_inst = fmin + df * t_sec
        
        # Calculate current based on instantaneous frequency
        return max_amplitude * np.sin(2 * np.pi * f_inst * t_sec)
    
    return current_func

## 2. Analysis with Different Current Amplitudes

Now let's analyze the model's response to zap currents with different amplitudes.

In [ ]:
# Simulation parameters
t_span = [0, 2000]  # 2000ms simulation (2 seconds)
dt = 0.02  # Time step in milliseconds
amplitudes = [0.1, 0.2]  # Current amplitudes in μA/cm²

# Create figure for plotting
fig, axes = plt.subplots(len(amplitudes) + 1, 1, figsize=(12, 4*len(amplitudes) + 2))
fig.suptitle('Zap Current Analysis', fontsize=14)

# First, plot the normalized current profile (only once)
t = np.arange(t_span[0], t_span[1], dt)
example_current = create_zap_current(1.0, 0, 80, t_span[1])
I_example = np.array([example_current(t_i) for t_i in t])
axes[0].plot(t/1000, I_example, 'k')
axes[0].set_ylabel('I_app (normalized)')
axes[0].set_yticks([-1, 0, 1])
axes[0].set_yticklabels(['-I_max', '0', 'I_max'])
axes[0].grid(True)

# Now simulate for each amplitude
for i, amp in enumerate(amplitudes, 1):
    model = HodgkinHuxleyModel()
    
    # Create zap current function
    current_func = create_zap_current(amp, 0, 80, t_span[1])
    
    # Simulate
    t, V, n, m, h = model.simulate(t_span, dt=dt, I_ext_func=current_func)
    
    # Plot membrane potential
    axes[i].plot(t/1000, V, 'k')
    axes[i].set_ylabel('Vm (mV)')
    axes[i].grid(True)
    axes[i].set_title(f'I_max = {amp} μA/cm²')
    
    # Set y-axis limits based on amplitude
    if amp < 0.15:
        axes[i].set_ylim(-74, -66)  # For subthreshold response
    else:
        axes[i].set_ylim(-85, 35)   # For suprathreshold response

# Set common x-label
axes[-1].set_xlabel('Time (sec)')

plt.tight_layout()
plt.savefig('../data/results/zap_analysis.png')
plt.show()

## 3. Frequency Response Analysis

Let's analyze the model's response at different frequencies by looking at the amplitude of voltage oscillations.

In [ ]:
def analyze_frequency_response(V, t, window_size=1000):
    """Analyze the amplitude of voltage oscillations in sliding windows.
    
    Args:
        V (array): Membrane potential values
        t (array): Time points
        window_size (int): Size of sliding window in time points
        
    Returns:
        tuple: Arrays of frequencies and corresponding amplitudes
    """
    n_windows = len(V) // window_size
    amplitudes = []
    frequencies = []
    
    for i in range(n_windows):
        start_idx = i * window_size
        end_idx = start_idx + window_size
        
        # Calculate mean frequency in window
        mean_time = np.mean(t[start_idx:end_idx]) / 1000.0  # Convert to seconds
        freq = fmin + (fmax - fmin) * mean_time / (t[-1] / 1000.0)
        
        # Calculate peak-to-peak amplitude
        amplitude = np.ptp(V[start_idx:end_idx])
        
        frequencies.append(freq)
        amplitudes.append(amplitude)
    
    return np.array(frequencies), np.array(amplitudes)

# Parameters for frequency analysis
fmin = 0
fmax = 80
window_size = int(1000 / dt)  # 1 second window

plt.figure(figsize=(10, 6))

for amp in amplitudes:
    model = HodgkinHuxleyModel()
    current_func = create_zap_current(amp, fmin, fmax, t_span[1])
    t, V, n, m, h = model.simulate(t_span, dt=dt, I_ext_func=current_func)
    
    freqs, amps = analyze_frequency_response(V, t, window_size)
    plt.plot(freqs, amps, '-o', label=f'I_max = {amp} μA/cm²')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Voltage Amplitude (mV)')
plt.title('Frequency Response')
plt.grid(True)
plt.legend()
plt.savefig('../data/results/frequency_response.png')
plt.show()